# using VGG16 model to extract features of a image and fine tuning VGG16

In [1]:
from os import listdir
from pickle import dump
import tensorflow as tf
import pandas as pd
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
import cv2
import numpy as np
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [11]:
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tqdm import tqdm
#R_LOSS_FACTOR = 10000
#import numpy as np

In [3]:
RUN_FOLDER='/home/pytorch_ashish/deep_learning/data/alt_text_data_all/models/NIC_model/'
new_model = load_model("/home/pytorch_ashish/deep_learning/data/alt_text_data_all/models/NIC_model/val_acc_9280_model.h5")

In [4]:
learning_rate=K.eval(new_model.optimizer.lr)
learning_rate

1.6e-07

In [2]:
image_dataframe=pd.read_csv('/home/pytorch_ashish/deep_learning/data/alt_text_data_all/image_location_venky.csv')
cengage= pd.read_csv('/home/pytorch_ashish/deep_learning/data/alt_text_data_all/cengage_production_data.csv')
pearson= pd.read_csv('/home/pytorch_ashish/deep_learning/data/alt_text_data_all/pearson_production_data.csv')
wiley=pd.read_csv('/home/pytorch_ashish/deep_learning/data/alt_text_data_all/wiley_production_data.csv')
wiley.rename(columns={'image_names':'Name'}, inplace=True)
pearson.rename(columns={'image_names':'Name'}, inplace=True)
cengage.rename(columns={'image_names':'Name'}, inplace=True)

image_dataframe=pd.merge(image_dataframe, cengage, on='Name', how='left')
image_dataframe.drop(['image_path', 'alt_text_html', 'Unnamed: 0'], axis=1, inplace=True)

image_dataframe=pd.merge(image_dataframe, pearson, on='Name', how='left')
image_dataframe["alt_text"] = image_dataframe["alt_text_x"].astype(str) + " "+image_dataframe["alt_text_y"].astype(str)
image_dataframe['subject_area'] = image_dataframe['subject_area_x'].astype(str) + " "+ image_dataframe['subject_area_y'].astype(str)
image_dataframe.drop(['alt_text_x', "alt_text_y", 'subject_area_x', 'subject_area_y', 'alt_text_html', 'image_path'], axis=1, inplace=True)

image_dataframe=pd.merge(image_dataframe, wiley, on='Name', how='left')
image_dataframe["alt_text"] = image_dataframe["alt_text_x"].astype(str) + " "+ image_dataframe["alt_text_y"].astype(str)
image_dataframe['subject_area'] = image_dataframe['subject_area_x'].astype(str) + " "+ image_dataframe['subject_area_y'].astype(str)
image_dataframe.drop(['alt_text_x', "alt_text_y", 'subject_area_x', 'subject_area_y', 'alt_text_html', 'image_path'], axis=1, inplace=True)

banned = ['nan']
f = lambda x: ' '.join([item for item in x.split() if item not in banned])
image_dataframe["subject_area"] = image_dataframe["subject_area"].apply(f)
image_dataframe["alt_text"] = image_dataframe["alt_text"].apply(f)
image_dataframe = image_dataframe[image_dataframe.subject_area != '']
image_dataframe.subject_area=image_dataframe.subject_area.replace({"other other": "others", "other": "others"})
image_dataframe.reset_index(drop=True, inplace=True)

include = ['maths', 'science', 'others', 'emss', 'canada', 'engineering', 'business', 'history']
indexNames=[]
for i in range(image_dataframe.shape[0]):
    if(image_dataframe.loc[i]['subject_area'] not in include):
        indexNames.append(i)
image_dataframe.drop(labels=indexNames, inplace=True)
image_dataframe.reset_index(drop=True, inplace=True)

In [3]:
image_dataframe['subject_area'].value_counts()

maths          146839
science         10820
others           5116
emss             3602
canada           2852
engineering      1854
business         1681
history          1014
Name: subject_area, dtype: int64

In [4]:
image_dataframe.subject_area.unique()

array(['maths', 'history', 'science', 'engineering', 'business', 'others',
       'emss', 'canada'], dtype=object)

In [5]:
image_dataframe.isna().sum()

Name            0
Path            0
Height          0
Width           0
alt_text        0
subject_area    0
dtype: int64

In [6]:
image_dataframe.shape

(173778, 6)

In [7]:
image_dataframe.head()

,Name,Path,Height,Width,alt_text,subject_area
0,9781337111348_11348_ch02_2.1_017-t2.png,/home/pytorch_ashish/deep_learning/data/alt_te...,131,181,The image consists of a Minitab output. Visual...,maths
1,9781285194790_9781285194790-551-t2.png,/home/pytorch_ashish/deep_learning/data/alt_te...,550,593,A map shows the Second Iraq War. Majority popu...,history
2,9781133947257_9781133947257_ch01_65-t2.png,/home/pytorch_ashish/deep_learning/data/alt_te...,101,184,In the Sherlock Holmes mystery The Final Solut...,maths
3,9781305652231_52231_ch01_f022-t2.png,/home/pytorch_ashish/deep_learning/data/alt_te...,24,188,The image consists of a number line. The numbe...,maths
4,ceng_image205.png,/home/pytorch_ashish/deep_learning/data/alt_te...,131,268,The image consists of a visual representation ...,maths


In [11]:
def gray(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_image = np.expand_dims(gray_image, axis=-1)
    return gray_image
BATCH_SIZE=32
trdata = ImageDataGenerator(rescale=1./255,  preprocessing_function=gray)
traindata = trdata.flow_from_dataframe(dataframe = image_dataframe
                                         , directory = None
                                         , x_col='Path'
                                         , y_col='subject_area'
                                         , target_size = (224,224)
                                         , batch_size = BATCH_SIZE
                                         , shuffle = False
                                         #, classes = y_train_en
                                         , class_mode = 'categorical'
                                            )


Found 173778 validated image filenames belonging to 8 classes.


In [12]:
int(round(image_dataframe.shape[0]/BATCH_SIZE ,0))

5431

In [13]:
image_dataframe.shape[0]/32

5430.5625

In [19]:
def extract_features_gen(dataframe):
   
    final_model = Model(inputs=new_model.inputs, outputs=new_model.layers[-2].output)
    # summarize
    print(final_model.summary())
   
    features = []
    for i in tqdm(range(int(round(dataframe.shape[0]/BATCH_SIZE ,0)))):
    
        images, labels = next(traindata)
        # get features
        feature = final_model.predict(images, verbose=0)
        
        features.extend(feature)
        
    features=np.array(features)
    return features

In [20]:
from tqdm import tqdm
features = extract_features_gen(image_dataframe)
print('Extracted Features: %d' % len(features))
RUN_FOLDER='/home/pytorch_ashish/deep_learning/data/alt_text_data_all/models/NIC_model/'
# save to file
dump(features, open(RUN_FOLDER+'features.pkl', 'wb'))

0%|          | 0/5431 [00:00<?, ?it/s]Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooli

In [21]:
features.shape

(173778, 4096)

In [9]:
def clean_sentence(sent):
	
    sent = sent.lower()
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    sent =sent.replace('x', 'x-axis')
    sent=sent.replace('y', 'y-axis')
    word_tokens = word_tokenize(sent)
    sent = ' '.join(w for w in word_tokens if not w in stop_words and len(w)>2)
    sent= re.sub(r"(\d+\s?)+","number ",sent)

    return sent


In [12]:
for i in tqdm(range(image_dataframe.shape[0])):
    clean=clean_sentence(image_dataframe.loc[i]['alt_text'])
    #print(clean)
    image_dataframe['alt_text'][i]='startseq ' + clean + ' endseq'

100%|██████████| 173778/173778 [09:21<00:00, 309.33it/s]


In [25]:
#image_dataframe['alt_text'][0]=clean
image_dataframe['alt_text'][11458]

'startseq attributes table underscore code underscore name underscore zip customer dot agent underscore code agent dot agent underscore code agent underscore phone line underscore code underscore name adares underscore zip customer dot agent underscore code number agent dot agent underscore code number agent underscore phone line underscore code underscore name rodriguez underscore zip customer dot agent underscore code number agent dot agent underscore code number agent underscore phone line underscore code underscore name rakowski underscore zip customer dot agent underscore code number agent dot agent underscore code number agent underscore phone line underscore code underscore name walker underscore zip customer dot agent underscore code number agent dot agent underscore code number agent underscore phone line underscore code underscore name vanloo underscore zip customer dot agent underscore code number agent dot agent underscore code number agent underscore phone line underscore 

In [26]:
image_dataframe.shape

(173778, 6)

In [ ]:
X = image_dataframe[0:]
